In [103]:
#Lets try to find scene for Utsira island, Norway
#first, I must know the istand's coordinates
#example coordinates for Utsira island
utsira_coords = [59.3081,4.8884]  # [longitude, latitude]
#example footprint of the island
utsira_footprint = {
    "type": "Polygon",
    "coordinates": [
        [            
            [59.285, 4.9333],
            [59.3381, 4.9333],
            [59.3281, 4.85],
            [59.25, 4.80],
            [59.285, 4.9333]
        ]
    ]
}
#lets check the cordinates are correct putting them on the map with folium
import folium  
# Create a map centered around Utsira island
utsira_map = folium.Map(location=utsira_coords, zoom_start=10)
# Add a marker for Utsira island
folium.Marker(location=utsira_coords, popup='Utsira Island').add_to(utsira_map)
# Add the footprint of the island
folium.Polygon(
    utsira_footprint['coordinates'][0],
    popup='Utsira Island Footprint',
    color='blue',
    fill=True,
    fill_color='blue',
    fill_opacity=0.2
).add_to(utsira_map)
#display the map
utsira_map

In [131]:
#ok, loks good
#lets try to find the scene for Utsira island in the STAC catalog
#STAC endpoint is https://eodatahub.org.uk/api/catalogue/stac/ but I will look in the airbus_pneo_data collection
import requests
import json
def find_scene_by_coordinates(bbox):
    url = "https://eodatahub.org.uk/api/catalogue/stac/search"
    params = {
        "datetime": "2020-01-31T23:00:00.000Z/2025-06-30T21:00:00.000Z", # specify the time range
        "collections": ["airbus_pneo_data"],
        "bbox": bbox,  # bbox in the format: min_lon,min_lat,max_lon,max_lat
        "limit": 100  # limit the number of results
    }
    # Print the parameters for debugging
    print("Request parameters:")
    # Use json.dumps to format the parameters nicely
    print (json.dumps(params))
    # Make the request to the STAC API
    response = requests.post(url, data=json.dumps(params))
    if response.status_code == 200:
        return response.json()
    else:
        return None
#find a bbox for the Utsira island footprint
#bbox is in the format: min_lon,min_lat,max_lon,max_lat
#mind the order of coordinates, it is [longitude, latitude] for bbox
utsira_bbox = [
    min(utsira_footprint['coordinates'][0], key=lambda x: x[1])[1],  # min longitude
    min(utsira_footprint['coordinates'][0], key=lambda x: x[0])[0],  # min latitude
    max(utsira_footprint['coordinates'][0], key=lambda x: x[1])[1],  # max longitude
    max(utsira_footprint['coordinates'][0], key=lambda x: x[0])[0]   # max latitude
]
print(utsira_bbox)
# Find scenes for Utsira island

utsira_scenes = find_scene_by_coordinates([utsira_coords[1], utsira_coords[0], utsira_coords[1], utsira_coords[0]])
# Check if scenes were found
if utsira_scenes and 'features' in utsira_scenes:
    print(f"Found {len(utsira_scenes['features'])} scenes for Utsira island:")
    for scene in utsira_scenes['features']:
        print(f"- {scene['id']}: {scene['properties']['datetime']}, {scene['bbox']}")
else:
    print("No scenes found for Utsira island.") 
utsira_scenes['features']

[4.8, 59.25, 4.9333, 59.3381]
Request parameters:
{"datetime": "2020-01-31T23:00:00.000Z/2025-06-30T21:00:00.000Z", "collections": ["airbus_pneo_data"], "bbox": [4.8884, 59.3081, 4.8884, 59.3081], "limit": 100}
Found 4 scenes for Utsira island:
- ACQ_PNEO3_04613512705534: 2024-07-20T11:25:08.309000Z, [4.6692073952, 59.1913895542, 5.0023782179, 59.3458178716]
- ACQ_PNEO4_04512012508170: 2024-07-06T11:06:32.902000Z, [4.7049321268, 59.1978341637, 4.9659157913, 59.3385997092]
- ACQ_PNEO4_04112011078659: 2024-03-24T11:06:25.118000Z, [4.7066702977, 59.2005253572, 4.963650134, 59.4524309482]
- ACQ_PNEO3_04128311050985: 2024-03-22T11:16:58.029000Z, [4.700912488, 59.1943790653, 4.9699318444, 59.4565772859]


[{'collection': 'airbus_pneo_data',
  'assets': {'external_quicklook': {'href': 'https://access.foundation.api.oneatlas.airbus.com/api/v1/items/9f4851b8-8c38-43e7-8e16-d27350761b09/quicklook',
    'type': 'application/octet-stream'},
   'quicklook': {'href': 'https://eodatahub.org.uk/api/catalogue/stac/catalogs/commercial/catalogs/airbus/collections/airbus_pneo_data/items/ACQ_PNEO3_04613512705534/quicklook',
    'type': 'application/octet-stream'},
   'external_thumbnail': {'href': 'https://access.foundation.api.oneatlas.airbus.com/api/v1/items/9f4851b8-8c38-43e7-8e16-d27350761b09/thumbnail',
    'type': 'application/octet-stream'},
   'thumbnail': {'href': 'https://eodatahub.org.uk/api/catalogue/stac/catalogs/commercial/catalogs/airbus/collections/airbus_pneo_data/items/ACQ_PNEO3_04613512705534/thumbnail',
    'type': 'application/octet-stream'}},
  'stac_version': '1.0.0',
  'id': 'ACQ_PNEO3_04613512705534',
  'bbox': [4.6692073952, 59.1913895542, 5.0023782179, 59.3458178716],
  'geo

In [142]:

utsira_map = folium.Map(location=utsira_coords, zoom_start=10)
def draw_scene(feature):
    """
    Draws a scene on the map using folium.
    """
    # Extract coordinates from the feature
    coords = feature['geometry']['coordinates'][0]
    coords_folium = [[coord[1], coord[0]] for coord in coords]
    print(coords)
    # Reverse the coordinates to [latitude, longitude] for folium
    print(coords_folium)
    # Add the scene footprint
    folium.Polygon(
        coords_folium,
        popup=f"{feature['id']} - {feature['properties']['datetime']}",
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.2
    ).add_to(utsira_map)

#iterate through the features and draw them on the map
for scene in utsira_scenes['features']:
    draw_scene(scene)



# Add a marker for Utsira island
folium.Marker(location=utsira_coords, popup='Utsira Island').add_to(utsira_map)

# Add the footprint of the island
folium.Polygon(
    utsira_footprint['coordinates'][0],
    popup='Utsira Island Footprint',
    color='blue',
    fill=True,
    fill_color='blue',
    fill_opacity=0.2
).add_to(utsira_map)

#display the map
utsira_map


[[5.0020001252, 59.3458178716], [4.6697406546, 59.3319769142], [4.6692073952, 59.1913895542], [5.0023782179, 59.2030051478], [5.0020001252, 59.3458178716]]
[[59.3458178716, 5.0020001252], [59.3319769142, 4.6697406546], [59.1913895542, 4.6692073952], [59.2030051478, 5.0023782179], [59.3458178716, 5.0020001252]]
[[4.9659157913, 59.3385997092], [4.7049321268, 59.3333544202], [4.7052381401, 59.1978341637], [4.9654798399, 59.2025102795], [4.9659157913, 59.3385997092]]
[[59.3385997092, 4.9659157913], [59.3333544202, 4.7049321268], [59.1978341637, 4.7052381401], [59.2025102795, 4.9654798399], [59.3385997092, 4.9659157913]]
[[4.9635617766, 59.4524309482], [4.7066702977, 59.4489627534], [4.7074783834, 59.3175672682], [4.7074804675, 59.3167561797], [4.7072827782, 59.2005253572], [4.9628329708, 59.2030984159], [4.9636478638, 59.3197553095], [4.963650134, 59.3205693276], [4.9635617766, 59.4524309482]]
[[59.4524309482, 4.9635617766], [59.4489627534, 4.7066702977], [59.3175672682, 4.7074783834], [59

In [207]:
#find the overlap of the scene 'ACQ_PNEO3_04613512705534' and the utsira footprint
#select the scene by id
scene_id = 'ACQ_PNEO3_04613512705534'
scene = next((s for s in utsira_scenes['features'] if s['id'] == scene_id), None)
if scene:
    print(f"Found scene {scene_id} with datetime {scene['properties']['datetime']} and bbox {scene['bbox']}")
else:
    print(f"Scene {scene_id} not found in the catalog.")    
print(f"Scene is: {scene['id']}")
print(f"Scene polygon original: {scene['geometry']}")
#compute the overlap of the scene geometry and the utsira footprint
from shapely.geometry import shape
from shapely.geometry import Polygon   
#convert the scene geometry to a shapely polygon
scene_shapely = scene
coords = scene_shapely['geometry']['coordinates'][0]
coords_shapely = [[coord[1], coord[0]] for coord in coords]
scene_shapely['geometry']['coordinates'][0] = coords_shapely
scene_polygon = shape(scene_shapely['geometry'])
#convert the utsira footprint to a shapely polygon
utsira_polygon = Polygon(utsira_footprint['coordinates'][0])
#compute the intersection of the two polygons
print(f"Scene polygon transformed: {scene_polygon}")
print(f"Utsira polygon: {utsira_polygon}")
intersection = scene_polygon.intersection(utsira_polygon)
#check if the intersection is empty
if intersection.is_empty:
    print(f"No overlap between the scene {scene_id} and the Utsira footprint.")
else:       
    print(f"Overlap found between the scene {scene_id} and the Utsira footprint.")
    print(f"Intersection area: {intersection.area}")
    print(f"Intersection coordinates: {intersection.exterior.coords[:]}")

#lets draw the intersection on the map
utsira_map = folium.Map(location=utsira_coords, zoom_start=10)
# Add a marker for Utsira island
folium.Marker(location=utsira_coords, popup='Utsira Island').add_to(utsira_map)
# Add the footprint of the intersection
# Convert intersection to a list of coordinates for folium
print(f"Intersection coordinates: {intersection.exterior.coords[:]}")
print(utsira_coords)
folium.Polygon(
    intersection.exterior.coords[:],
    popup='Utsira Island Footprint and scene intersection',
    color='blue',
    fill=True,
    fill_color='blue',
    fill_opacity=0.2
).add_to(utsira_map)

utsira_map


Found scene ACQ_PNEO3_04613512705534 with datetime 2024-07-20T11:25:08.309000Z and bbox [4.6692073952, 59.1913895542, 5.0023782179, 59.3458178716]
Scene is: ACQ_PNEO3_04613512705534
Scene polygon original: {'type': 'Polygon', 'coordinates': [[[5.0020001252, 59.3458178716], [4.6697406546, 59.3319769142], [4.6692073952, 59.1913895542], [5.0023782179, 59.2030051478], [5.0020001252, 59.3458178716]]]}
Scene polygon transformed: POLYGON ((59.3458178716 5.0020001252, 59.3319769142 4.6697406546, 59.1913895542 4.6692073952, 59.2030051478 5.0023782179, 59.3458178716 5.0020001252))
Utsira polygon: POLYGON ((59.285 4.9333, 59.3381 4.9333, 59.3281 4.85, 59.25 4.8, 59.285 4.9333))
Overlap found between the scene ACQ_PNEO3_04613512705534 and the Utsira footprint.
Intersection area: 0.006541980000000076
Intersection coordinates: [(59.3381, 4.9333), (59.3281, 4.85), (59.25, 4.8), (59.285, 4.9333), (59.3381, 4.9333)]
Intersection coordinates: [(59.3381, 4.9333), (59.3281, 4.85), (59.25, 4.8), (59.285, 4

In [210]:
#generate url https://eodatahub.org.uk/api/catalogue/stac/catalogs/commercial/catalogs/airbus/collections/airbus_sar_data/items/TSX-1_SM_S_strip_014R_93272_A14896910_8000/quote
#lets try to get the quote for the scene
import requests 
def get_quote(scene_id, coordinates, date_range, license):
    url = f"https://eodatahub.org.uk/api/catalogue/stac/catalogs/commercial/catalogs/airbus/collections/airbus_sar_data/items/{scene_id}/quote"
    body = {"coordinates":coordinates,
            "date_range":date_range,
            "productBundle": "General Use",
            "licence":license}
    response = requests.post(url,data=json.dumps(body))
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching quote for scene {scene_id}: {response.status_code}")
        return None
# Get the quote for the scene
scene_quote = get_quote(scene_id,intersection.exterior.coords[:], "2020-01-31T23:00:00.000Z/2025-06-30T21:00:00.000Z", "CC-BY-SA-4.0")
scene_quote

Error fetching quote for scene ACQ_PNEO3_04613512705534: 504


In [211]:
#generate url https://eodatahub.org.uk/api/catalogue/stac/catalogs/commercial/catalogs/airbus/collections/airbus_sar_data/items/TSX-1_SM_S_strip_014R_93272_A14896910_8000/order
#lets try to get the quote for the scene
import requests 

___delete this before ordering___
def order_pneo(scene_id, coordinates, date_range, license):
    url = f"https://_eodatahub.org.uk/api/catalogue/stac/catalogs/commercial/catalogs/airbus/collections/airbus_sar_data/items/{scene_id}/quote"
    body = {"coordinates":coordinates,
            "date_range":date_range,
            "productBundle": "General Use",
            "licence":license}
    response = requests.post(url,data=json.dumps(body))
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching quote for scene {scene_id}: {response.status_code}")
        return None
# Get the quote for the scene
scene_quote = order_pneo(scene_id,intersection.exterior.coords[:], "2020-01-31T23:00:00.000Z/2025-06-30T21:00:00.000Z", "CC-BY-SA-4.0")
scene_quote

SyntaxError: invalid syntax (4046121796.py, line 5)